<a href="https://colab.research.google.com/github/pranav-vijayananth/SOMResearch/blob/main/CPU_GPU_tensorflow_som.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!wget https://raw.githubusercontent.com/cgorman/tensorflow-som/master/tf_som.py

In [ ]:
!pip3 install tensorflow==1.15

In [ ]:
import numpy as np

np.set_printoptions(suppress=True)

import math
import tensorflow as tf
from tf_som import SelfOrganizingMap
from scipy.spatial import distance_matrix
from collections import Counter
import logging
import subprocess
import time
from tqdm import tqdm

print(tf.__version__)

In [ ]:
print(tf.test.gpu_device_name())

In [ ]:
# GETTING LABELS FROM PREDICTION

def get_labels(prediction):
    labels = []
    
    for arr in prediction:
        c=0
        for label in arr:
            c+=1
            if(c==2):
                labels.append(label)
    
    labels = ",".join(list(map(lambda item: str(item), labels)))
    return labels

In [ ]:
#HEAT MAP OF CLUSTERS AND BMU LOCATIONS

def get_umatrix(input_vects, weights, m, n):
    umatrix = np.zeros((m * n, 1))
    neuron_locs = list()
    for i in range(m):
        for j in range(n):
            neuron_locs.append(np.array([i, j]))

    neuron_distmat = distance_matrix(neuron_locs, neuron_locs)

    for i in range(m * n):
        neighbor_idxs = neuron_distmat[i] <= 1
        # print("index: ",neighbor_idxs)
        # print("shape: ",weights.shape)
        neighbor_weights = weights[neighbor_idxs]
        umatrix[i] = distance_matrix(np.expand_dims(weights[i], 0), neighbor_weights).mean()

    bmu_indices = []
    for vect in input_vects:
        min_index = min([i for i in range(len(list(weights)))],
                        key=lambda x: np.linalg.norm(vect-
                                                     list(weights)[x]))
        bmu_indices.append(neuron_locs[min_index])
        
    return umatrix, bmu_indices

In [ ]:
path = "/content/drive/MyDrive/Datasets/Research/SelectedDatasets/veteran.csv"

files = path.split("/")[-1]
name = files.split(".")[0]

print(name)

filename = open(path)

data = []
groundtruth = []
dataread = filename.readline()
dataread = filename.readline()

while dataread != "": 
    a = dataread.split(",")
    l2 = []
    for j in range(0, len(a), 1):
        if j == len(a)-1:
            groundtruth.append(a[j].strip())
        else:
            try: 
                l2.append(float(a[j]))
            except:
                l2.append(0)
    data.append(l2)
    dataread = filename.readline()

rows = len(data)
cols = len(data[0])
num_inputs = rows * cols

print(f"Rows: {rows}")
print(f"Columns: {cols}")
print(f"Neurons: {num_inputs}")

filename.close()

In [ ]:
#CLUSTERS

uniqueValues = Counter(groundtruth).keys()
clusters = len(uniqueValues)
print(f"The clusters in dataset: {clusters}")

In [ ]:
start = time.time()

for i in tqdm(range(1, 100+1)):
  #CREATING THE TENSORFLOW GRAPH AND LOGS FOR THE MODEL
        
  logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
  graph = tf.Graph()

  with graph.as_default():
      session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(
                  allow_soft_placement=True,
                  log_device_placement=False
      ))

      #BATCH SIZE 
      batch_size = 32
      
      #CONVERT TO NUMPY ARRAY FOR TF DATA PIPELINE
      data = np.array(data, dtype="float32")
      
      #MAKING THE TENSORFLOW DATRASET PIPELINE 
      input_data = tf.compat.v1.data.Dataset.from_tensor_slices(data)
      input_data = input_data.repeat()
      input_data = input_data.batch(batch_size)
      iterator = input_data.make_one_shot_iterator()
      next_element = iterator.get_next()
      
      #BUILDING THE SOM OBJECT
      som = SelfOrganizingMap(m=1, n=clusters, dim=cols, max_epochs=20, session=session, graph=graph, input_tensor=next_element, batch_size=batch_size, initial_learning_rate=0.1,gpus=1, model_name='Self-Organizing-Map', softmax_activity=True)
      
      #MAKING + RUNNING SESSION
      init_op = tf.compat.v1.global_variables_initializer()
      session.run([init_op])
      
      #TRAINING
      som.train(num_inputs)
      
      #SOM UMATRIX + BMU LOCATION/PREDICTION
      umatrix, bmu_loc = get_umatrix(data, som.output_weights, 1, clusters)
      labels = get_labels(bmu_loc)

      bash_command = f"touch /content/drive/MyDrive/Datasets/Research/GPU+CPULabels/CPU/{name}/{name}-{i}.txt"
      subprocess.run(bash_command, shell=True, check=True)

      fo = open(f"/content/drive/MyDrive/Datasets/Research/GPU+CPULabels/CPU/{name}/{name}-{i}.txt", "a")
      fo.write(labels)
      fo.write("\n")
      
      fo.close()

stop = time.time()
print(f"{files} / CPU / {abs(start-stop)} ")
      